# Gesture UI Project 1
By Levente Kalman

Student no: G00392201

In [ ]:
# Imports
import os
import glob 
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [3]:
# Set random seed 
RANDOM_SEED = 392201

In [ ]:
# Define dataset path
DATASET_PATH = sorted(glob.glob('fashion/*/*.png'))

# Define the actual categories present in the dataset
categories = ['ankleboot', 'bag', 'coat', 'dress', 'pullover', 'sandal', 'shirt', 'sneaker', 'trouser', 'tshirt-top']

# Create a mapping from class names to integer labels
class_mapping = {category: idx for idx, category in enumerate(categories)}

# Create a file list
X = np.array([np.array(Image.open(f).convert('L').resize((20, 20))).flatten() for f in DATASET_PATH]) / 255.0 # normalize pixels
# Extract class labels from filenames
y = np.array([class_mapping[os.path.basename(os.path.dirname(f))] for f in DATASET_PATH])

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("First 10 labels:", y[:10])  # Check labels

a = [s.split() for s in " ".join(DATASET_PATH).split("\\") if s]

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=RANDOM_SEED)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_SEED)

# Print shapes of datasets
print("Training set size:", X_train.shape, y_train.shape)
print("Validation set size:", X_valid.shape, y_valid.shape)
print("Test set size:", X_test.shape, y_test.shape)

Shape of X: (70000, 400)
Shape of y: (70000,)
First 10 labels: [0 0 0 0 0 0 0 0 0 0]
Training set size: (42000, 400) (42000,)
Validation set size: (14000, 400) (14000,)
Test set size: (14000, 400) (14000,)


# Classifier Methods
Classification algorithms used:
- SVM
- Logistical Regression
- K Nearest Neighbor
- Random Forest

## SVM 

In [ ]:
# Initialize the SVM classifier
svm_classifier = SVC(random_state=RANDOM_SEED)

# Train the classifier
svm_classifier.fit(X_train, y_train)

# Evaluate the classifier on the validation set
y_pred = svm_classifier.predict(X_valid)
print(classification_report(y_valid, y_pred, target_names=categories))
print(confusion_matrix(y_valid, y_pred))